# HurdleDMR from Python

HurdleDMR.jl is a Julia implementation of the Hurdle Distributed Multinomial Regression (HDMR), as described in:

[Bryan Kelly, Asaf Manela & Alan Moreira (2021)](https://doi.org/10.1080/07350015.2021.1947843) Text Selection, Journal of Business & Economic Statistics [(ungated preprint)](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3491942).

It includes a Julia implementation of the Distributed Multinomial Regression (DMR) model of [Taddy (2015)](https://arxiv.org/abs/1311.6139).

This tutorial explains how to use this package from Python via the [PyJulia](https://github.com/JuliaPy/pyjulia) package.

## Setup

### Install Julia

First, install Julia itself. The easiest way to do that is to get the latest stable release from the official [download page](https://julialang.org/downloads/). An alternative is to install [JuliaPro](https://juliacomputing.com/products/juliapro/).

Once installed, open julia in a terminal (or in Juno), press `]` to activate package manager and add the following packages:
```
pkg> add HurdleDMR GLM Lasso
```

### Install PyJulia
See the documentation [here](https://pyjulia.readthedocs.io/en/stable/) for installation instructions.

In [1]:
from julia.api import Julia
jl = Julia(compiled_modules=False)

`jl` allows us to evaluate julia code.

### Example Data

The data should either be an n-by-p covars matrix or a DataFrame containing the covariates, and a (sparse) n-by-d counts matrix.

For illustration we'll analyse the State of the Union text that is roughly annual and relate it to stock market returns.

The `sotu.jl` script compiles stock market execss returns and the State of the Union Address texts into a matching DataFrame `covarsdf` and a sparse document-term matrix `counts`.

In [2]:
jl.eval('import Pkg, PyCall')
jl.eval('Pkg.add(["Lasso", "CSV", "DataDeps", "DataFrames", "Pandas", "FamaFrenchData", "TextAnalysis", "SparseArrays", "PyCall"])')
covarsdf, counts, terms = jl.eval('include("sotu.jl")')

If it throws any errors about missing packages, go ahead and add them and run again.
The following code block converts the counts matrix and dataframe into python types which is probably what you start with if you work in python.

In [3]:
import numpy as np
from scipy import sparse
import pandas as pd
from julia import Main, Pandas, DataFrames, SparseArrays, Lasso

pycounts = sparse.csc_matrix(counts)
pycovarsdf = Pandas.DataFrame(covarsdf)

`pycounts` now holds the sparse matrix of bigram counts from the text:

In [4]:
pycounts

<92x163 sparse matrix of type '<class 'numpy.int64'>'
	with 3376 stored elements in Compressed Sparse Column format>

`pycovarsdf` contains the non-text data we'll use for this example:

In [5]:
pycovarsdf

,Rem,Date,President
0,29.47,1927,Calvin Coolidge
1,35.39,1928,Calvin Coolidge
2,-19.54,1929,Herbert Hoover
3,-31.23,1930,Herbert Hoover
4,-45.11,1931,Herbert Hoover
...,...,...,...
87,0.08,2015,Barack Obama
88,13.30,2016,Barack Obama
89,21.51,2017,Donald J. Trump
90,-6.93,2018,Donald J. Trump


### Add parallel workers and make HurdleDMR package available to workers

In [6]:
jl.eval("using Distributed")
from julia.Distributed import addprocs
addprocs(4)

from julia import HurdleDMR as hd
jl.eval("@everywhere using HurdleDMR")

From now on the HurdleDMR package's functions can be called from within python using the `hd` alias.

## Distributed Multinomial Regression (DMR)

The Distributed Multinomial Regression (DMR) model of Taddy (2015) is a highly scalable
approximation to the Multinomial using distributed (independent, parallel)
Poisson regressions, one for each of the d categories (columns) of a large `counts` matrix,
on the `covarsdf`.

To fit a DMR:

In [7]:
covars = np.asmatrix(pycovarsdf[['Rem']])

In [8]:
jlcounts = Main.scipyCSC_to_julia(pycounts)

In [9]:
m = hd.dmr(covars, jlcounts)

or with a dataframe and formula, by first converting the pandas dataframe to julia

In [10]:
jlcovarsdf = Main.pd_to_df(pycovarsdf)

and then fitting a DMR model that uses this DataFrame and a specified model

In [11]:
mf = jl.eval('@model(c ~ Rem)')
m = hd.fit(hd.DMR, mf, jlcovarsdf, jlcounts)

We can get the coefficients matrix for each variable + intercept as usual with

In [12]:
hd.coef(m)

array([[-5.43397777e+00, -5.16903423e+00, -5.48346231e+00,
        -5.87732380e+00, -5.87379264e+00, -5.77114439e+00,
        -3.23543281e+00, -5.38167962e+00, -4.41075981e+00,
        -5.67893791e+00, -5.60338003e+00, -5.01658441e+00,
        -3.90873484e+00, -5.72061820e+00, -5.73630459e+00,
        -5.72462434e+00, -5.66892541e+00, -5.38138811e+00,
        -4.80727621e+00, -5.66217618e+00, -5.48220062e+00,
        -5.92779117e+00, -5.84191010e+00, -5.65463036e+00,
        -5.35306350e+00, -5.72462432e+00, -5.36394515e+00,
        -5.26031876e+00, -5.25619797e+00, -5.86001501e+00,
        -4.94219746e+00, -5.49505171e+00, -5.49520004e+00,
        -5.47838157e+00, -5.55757029e+00, -5.08246018e+00,
        -5.32896413e+00, -4.32989185e+00, -5.75678995e+00,
        -5.56106908e+00, -5.31207341e+00, -6.05074799e+00,
        -5.68017255e+00, -5.15241608e+00, -5.67987417e+00,
        -3.34760668e+00, -5.78230946e+00, -5.49833280e+00,
        -5.17808067e+00, -5.01110532e+00, -5.83195551e+0

By default we only return the AICc maximizing coefficients.
To also get back the entire regulatrization paths, run

In [13]:
paths = hd.dmrpaths(covars, jlcounts)

We can now select, for example the coefficients that minimize 10-fold CV mse (takes a little longer)

In [14]:
jl.eval("using Lasso: MinCVmse")
segselect = jl.eval("MinCVKfold{MinCVmse}(10)")
hd.coef(paths, segselect)

array([[-5.44837099e+00, -5.23214789e+00, -5.43167163e+00,
        -5.77114434e+00, -5.81993469e+00, -5.77114439e+00,
        -3.23543281e+00, -5.38167962e+00, -4.32703043e+00,
        -5.72462437e+00, -5.59797816e+00, -5.01658441e+00,
        -3.97147974e+00, -5.72061820e+00, -5.72462437e+00,
        -5.72462434e+00, -5.66357350e+00, -5.34993092e+00,
        -4.80799960e+00, -5.72462437e+00, -5.34993085e+00,
        -5.81993455e+00, -5.81453663e+00, -5.68017261e+00,
        -5.31915926e+00, -5.72462432e+00, -5.34993090e+00,
        -5.26031876e+00, -5.25735906e+00, -5.81993460e+00,
        -4.93049270e+00, -5.55757029e+00, -5.55757029e+00,
        -5.34993092e+00, -5.55757029e+00, -5.04121497e+00,
        -5.34993092e+00, -4.32989185e+00, -5.75678995e+00,
        -5.48346231e+00, -5.34993092e+00, -6.05074799e+00,
        -5.68017255e+00, -5.15241608e+00, -5.63761301e+00,
        -3.34760668e+00, -5.72462437e+00, -5.44837111e+00,
        -5.17808067e+00, -4.98702548e+00, -5.72462437e+0

## Hurdle Distributed Multinomial Regression (HDMR)

For highly sparse counts, as is often the case with text that is selected for
various reasons, the Hurdle Distributed Multinomial Regression (HDMR) model of
Kelly, Manela, and Moreira (2021), may be superior to the DMR. It approximates
a higher dispersion Multinomial using distributed (independent, parallel)
Hurdle regressions, one for each of the d categories (columns) of a large `counts` matrix,
on the `covars`. It allows a potentially different sets of covariates to explain
category inclusion ($h=1{c>0}$), and repetition ($c>0$) using the optional `inpos` and `inzero` keyword arguments.

Both the model for zeroes and for positive counts are regularized by default,
using `GammaLassoPath`, picking the AICc optimal segment of the regularization
path.

HDMR can be fitted:

In [15]:
m = hd.hdmr(covars, jlcounts)

We can get the coefficients matrix for each variable + intercept as usual though now there is a set of coefficients for the model for repetitions and for inclusions

In [16]:
coefspos, coefszero = hd.coef(m)
print("Repetition coefficients:\n", coefspos)
print("Inclusion coefficients:\n", coefszero)

Repetition coefficients:
 [[-4.10095408e+00 -2.46311402e+00 -5.69913212e+00 -5.81114099e+00
  -4.56226268e+00 -5.65314535e+00 -2.83891033e+00 -4.73221532e+00
  -3.87792475e+00 -4.44134321e+00 -4.13831405e+00 -4.46968430e+00
  -2.67566592e+00 -6.17378610e+00 -5.04407514e+00 -5.77337923e+00
  -4.28880607e+00 -3.76699723e+00 -3.91687016e+00 -4.71836406e+00
  -2.75060981e+00 -4.19269956e+00 -5.06164401e+00 -4.00593360e+00
  -4.61806297e+00 -5.86929691e+00 -4.94799149e+00 -4.54616070e+00
  -4.86192434e+00 -5.56922079e+00 -3.13386424e+00 -3.86610236e+00
  -4.69262832e+00 -3.40511128e+00 -4.77222427e+00 -2.60553464e+00
  -4.68293824e+00 -3.64773029e+00 -5.37909996e+00 -4.51931403e+00
  -4.90640673e+00 -5.56500032e+00 -5.61682539e+00 -5.16216290e+00
  -4.80322994e+00 -2.64920181e+00 -6.15832660e+00 -4.70545239e+00
  -3.85762893e+00 -4.35065915e+00 -4.75062052e+00 -4.59123707e+00
  -3.86745469e+00 -5.00864540e+00 -3.01405000e+00 -3.17470691e+00
  -4.44428881e+00 -5.03173570e+00 -4.97399211e+00 

By default we only return the AICc maximizing coefficients.
To get the coefficients that minimize say the BIC criterion, run

In [17]:
paths = hd.hdmrpaths(covars, jlcounts)

coefspos, coefszero = hd.coef(paths, Lasso.MinBIC())

In [18]:
coefspos # repetition coefs

array([[-4.10095408e+00, -2.46311402e+00, -5.69913212e+00,
        -5.81114099e+00, -4.56226268e+00, -5.65314535e+00,
        -2.83891033e+00, -4.73221532e+00, -3.87792475e+00,
        -4.44134321e+00, -4.13831405e+00, -4.46968430e+00,
        -2.67566592e+00, -6.17378610e+00, -5.04407514e+00,
        -5.77337923e+00, -4.28880607e+00, -3.76699723e+00,
        -3.91687016e+00, -4.71836406e+00, -2.75060981e+00,
        -4.19269956e+00, -5.06164401e+00, -4.04950441e+00,
        -4.61806297e+00, -5.86929691e+00, -4.94799149e+00,
        -4.50407402e+00, -4.86192434e+00, -5.56922079e+00,
        -3.13386424e+00, -3.86610236e+00, -4.69262832e+00,
        -3.40511128e+00, -4.77222427e+00, -4.03006730e+00,
        -4.68293824e+00, -3.64773029e+00, -5.37909996e+00,
        -4.51931403e+00, -4.90640673e+00, -5.56500032e+00,
        -5.61682539e+00, -5.16216290e+00, -4.80322994e+00,
        -2.64920181e+00, -6.15832660e+00, -4.70545239e+00,
        -3.85762893e+00, -4.35065915e+00, -4.75062052e+0

In [19]:
coefszero # inclusion coefs

array([[-5.91841847e-01, -9.31126067e-01,  3.30163123e-02,
        -3.77458483e-01, -4.78606926e-01, -6.75867641e-02,
         3.20423878e+00,  2.05630060e-01,  9.63012435e-01,
        -5.91841847e-01, -9.56833515e-01,  3.82168936e-01,
         3.95854018e-01, -3.48163357e-02, -3.28946838e-01,
        -1.62505868e-01, -7.90407397e-01, -3.45020538e-01,
         4.29928449e-01, -6.87688948e-01, -1.11938166e+00,
        -7.90407397e-01, -4.94693837e-01, -8.53276186e-01,
         1.08858988e-01, -1.60684409e-01, -5.33873182e-02,
         8.17912836e-02,  2.61762473e-01, -3.58464942e-01,
        -1.01712931e-01, -8.79982841e-01, -4.04644809e-01,
        -5.71433231e-01, -3.60035703e-01, -2.63897256e+00,
        -5.01823012e-01,  1.25918958e+00, -5.01823012e-01,
        -3.23351926e-01,  2.01881344e-01, -6.42878566e-01,
        -6.96929692e-02,  5.15354465e-01, -5.89831278e-01,
         1.88071328e+00, -5.91841847e-01, -4.76079700e-02,
         8.70457369e-02,  4.29928449e-01, -7.90407397e-0

## Sufficient reduction projection

A sufficient reduction projection summarizes the counts, much like a sufficient
statistic, and is useful for reducing the d dimensional counts in a potentially
much lower dimension matrix `z`.

To get a sufficient reduction projection in direction of `Rem` for the above
example

In [20]:
hd.srproj(m,jlcounts,1,1)

array([[ 2.81751428e-02, -1.15931458e-04,  3.00000000e+01,
         2.20000000e+01],
       [ 1.64189960e-02, -8.05878853e-04,  2.80000000e+01,
         3.30000000e+01],
       [-1.01566985e-03, -5.93850167e-03,  4.40000000e+01,
         3.40000000e+01],
       [-2.35867899e-03, -1.15558569e-02,  1.00000000e+01,
         1.70000000e+01],
       [-4.79357759e-02, -9.79303560e-03,  1.70000000e+01,
         2.20000000e+01],
       [-2.38473991e-02, -4.09846822e-03,  1.30000000e+01,
         2.00000000e+01],
       [ 1.83287115e-02,  8.25493723e-04,  5.00000000e+00,
         1.40000000e+01],
       [ 1.11599797e-02,  2.22875151e-03,  6.00000000e+00,
         1.40000000e+01],
       [ 1.58260699e-02,  1.62366624e-03,  8.00000000e+00,
         1.90000000e+01],
       [ 1.26266272e-02, -6.98256013e-03,  4.00000000e+00,
         1.70000000e+01],
       [ 2.92400211e-02,  1.84979841e-03,  2.10000000e+01,
         1.90000000e+01],
       [ 1.06612616e-02,  1.76424265e-04,  1.40000000e+01,
      

Column 1 is `zpos`: the SR projection summarizing the information in repeated use of terms.

Column 2 is `zzero`: the SR projection summarizing the information in term inclusion.

Column 3 is `m`: the total number of excess counts.

Column 4 is `ℓ`: the total number of nonzero counts.

## Counts Inverse Regression (CIR)

Counts inverse regression allows us to predict a covariate with the counts and other covariates.
Here we use hdmr for the backward regression and another model for the forward regression.
This can be accomplished with a single command, by fitting a CIR{HDMR,FM} where the forward model is FM <: RegressionModel.

In [21]:
jl.eval("using GLM: LinearModel")
spec = jl.eval("CIR{HDMR,LinearModel}")
mf = jl.eval("@model(h ~ President + Rem, c ~ President + Rem)")
cir = hd.fit(spec, mf, jlcovarsdf, jlcounts, "Rem", nocounts=True)
cir

<PyCall.jlwrap TableCountsRegressionModel{CIR{HDMR,LinearModel},DataFrame,SparseMatrixCSC{Float64,Int64}}

2-part model: [h ~ President + Rem, c ~ President + Rem]

Forward model coefficients for predicting Rem:
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                    Coef.    Std. Error      t  Pr(>|t|)    Lower 95%      Upper 95%
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
(Intercept)                                    17.4837       11.3688      1.54    0.1285    -5.17963      40.147
President: PyObject 'Calvin Coolidge'          13.0088       13.4688      0.97    0.3374   -13.8407       39.8584
President: PyObject 'Donald J. Trump'          -2.14014      11.22       -0.19    0.8493   -24.5069       20.2266
President: PyObject 'Dwight D. Eisenhower'    -10.4126        8.31052    -1.25    0.2143   -26.9

where the ```nocounts=True``` means we also fit a benchmark model without counts. 
The last few coefficients are due to text data.
`zpos` is the SR projection summarizing the information in repeated use of terms.
`zzero` is the SR projection summarizing the information in term inclusion.
`m` is the total number of excess counts.
`ℓ` is the total number of nonzero counts.

We can get the forward and backward model coefficients with

In [22]:
hd.coefbwd(cir)

(array([[-7.50577915e+00, -3.28272336e+00, -5.62136487e+00, ...,
          0.00000000e+00,  0.00000000e+00, -3.84224956e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        ...,
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00, -2.40875556e-02, ...,
          0.00000000e+00,  0.00000000e+00,  7.22724288e-03]]),
 array([[-0.59184185,  2.26270697,  0.04011288, ...,  0.        ,
          0.        ,  1.18279386],
        [ 0.        , -4.75743505,  0.        , ...,  0.        ,
          0.        ,  0.91905273],
        [ 0.        , -5.

In [23]:
hd.coeffwd(cir)

array([ 1.74836878e+01,  1.30088423e+01, -2.14014450e+00, -1.04126228e+01,
       -6.29262698e+00, -2.22441239e+01, -2.02674778e+01, -5.36948700e+00,
       -9.36384785e+00, -3.61277714e+01, -1.42189166e+01, -1.84066861e+01,
       -1.80297494e+01, -3.37794987e+01, -1.67344371e+01, -3.09841666e+00,
        4.77130987e+02,  9.00265291e+03, -5.06128436e-02,  3.67346567e-02])

The fitted model can be used to predict vy with new data

In [24]:
jlcovarsnewdata = Main.pd_to_df(pycovarsdf.iloc[range(0,10), :])
jlcountsnewdata = Main.scipyCSC_to_julia(pycounts[range(0,10), :])

In [25]:
hd.predict(cir, jlcovarsnewdata, jlcountsnewdata)

[35.01327624412601,
 29.846723755873985,
 -21.60376552127853,
 -24.869433502443925,
 -33.695746592945596,
 -25.101054383331906,
 11.735827128269902,
 19.331781366167014,
 17.275545003481156,
 10.8159318912295]

We can also predict only with the other covariates, which in this case
is just a linear regression

In [26]:
hd.predict(cir, jlcovarsnewdata, jlcountsnewdata, nocounts=True)

[32.43000000000001,
 32.43000000000001,
 -26.317499999999995,
 -26.317499999999995,
 -26.317499999999995,
 -26.317499999999995,
 13.509166666666667,
 13.509166666666667,
 13.509166666666667,
 13.509166666666667]

Kelly, Manela, and Moreira (2021) show that the differences between DMR and HDMR can be substantial in some cases, especially when the counts data is highly sparse.

Please reference the paper for additional details and example applications.